In [2]:
!pip install numba==0.48

In [3]:
!pip install regain

In [4]:
%matplotlib inline
import time
import numpy as np
import pandas as pd

# Sample data

In [5]:
X = np.array([[0.8, 0.0, 0.2, 0.0],
                     [0.0, 0.4, 0.0, 0.0],
                     [0.2, 0.0, 0.3, 0.1],
                     [0.0, 0.0, 0.1, 0.7]])

In [6]:
# np.random.seed(243)
# X = np.random.multivariate_normal(mean=[0, 0, 0, 0], cov=true_cov, size=200)
# X

# Best model solution Z

Assume found parameters with GridSearch

In [7]:
import sys
sys.path.append('..')
# sys.path.append(sys.path[0] + "/..")

from gglasso.solver.single_admm_solver import ADMM_SGL

In [10]:
Omega_0 = np.eye(len(X))
lambda1 = 0.01

In [12]:
sol, info = ADMM_SGL(X, lambda1, Omega_0 , tol = 1e-10, verbose = False, latent = False)

ADMM terminated after 170 iterations with accuracy 2.3682787500348695e-05
ADMM status: {'primal optimal', 'dual optimal'}


In [13]:
Z = np.array(sol["Theta"])

# Import sklearn

In [14]:
from sklearn.covariance import GraphicalLasso
from sklearn.covariance import empirical_covariance

In [15]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    rg = GraphicalLasso(tol=tol, max_iter = 1000).fit(X)
    end = time.time()
    cov_list.append(rg.covariance_)
    precision_list.append(rg.precision_)
    time_list.append(end-start)

In [16]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [17]:
method_list = ["sklearn"]*len(accuracy_list)

In [18]:
df_sk = pd.DataFrame(data={'time': time_list, 
                           'accuracy': accuracy_list,
                           'method':method_list})
df_sk.head()

,time,accuracy,method
0,0.005095,-96.418703,sklearn
1,0.002090,-96.418703,sklearn
2,0.001961,-96.418731,sklearn
3,0.002428,-96.418735,sklearn


# Import Regain GL

In [19]:
from regain.covariance import GraphicalLasso
#from regain.covariance.graphical_lasso_ import GraphicalLasso

In [20]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    rg = GraphicalLasso(alpha=0.01, rho=1.,tol=tol, rtol=1e-7, max_iter=1000, verbose=False).fit(X)
    end = time.time()
    cov_list.append(rg.covariance_)
    precision_list.append(rg.precision_)
    time_list.append(end-start)

In [21]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [22]:
method_list = ["regain"]*len(accuracy_list)

In [23]:
df_rg = pd.DataFrame(data={'time': time_list, 
                           'accuracy': accuracy_list,
                           'method':method_list})
df_rg.head()

,time,accuracy,method
0,0.059809,-99.119188,regain
1,0.052288,-96.704805,regain
2,0.059125,-96.447454,regain
3,0.077772,-96.421612,regain


# Import ADMM SGL

In [24]:
#from helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix

In [25]:
import sys
sys.path.append('..')
# sys.path.append(sys.path[0] + "/..")

from gglasso.solver.single_admm_solver import ADMM_SGL

In [26]:
Omega_0 = np.eye(X.shape[0])
lambda1 = 0.01

In [28]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    sol, info = ADMM_SGL(X, lambda1, Omega_0 , tol = tol, verbose = False, latent = False)
    end = time.time()
    cov_list.append(sol['Omega'])
    precision_list.append(sol['Theta'])
    time_list.append(end-start)

ADMM terminated after 92 iterations with accuracy 0.0010125889165137512
ADMM status: {'primal optimal', 'dual optimal'}
ADMM terminated after 136 iterations with accuracy 0.00012137032648511202
ADMM status: {'primal optimal', 'dual optimal'}
ADMM terminated after 163 iterations with accuracy 3.315146140899157e-05
ADMM status: {'primal optimal', 'dual optimal'}
ADMM terminated after 169 iterations with accuracy 2.4848445583855884e-05
ADMM status: {'primal optimal', 'dual optimal'}


In [29]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [30]:
method_list = ["admm"]*len(accuracy_list)

In [31]:
df_admm = pd.DataFrame(data={'time': time_list, 
                             'accuracy': accuracy_list,
                             'method':method_list})
df_admm.head()

,time,accuracy,method
0,0.014794,0.979997,admm
1,0.017387,0.998016,admm
2,0.018357,0.999808,admm
3,0.021240,0.999976,admm


# Plotting

In [32]:
# !pip install plotly

In [45]:
frames = [df_sk, df_rg, df_admm]

df = pd.concat(frames)
df

,time,accuracy,method
0,0.005095,-96.418703,sklearn
1,0.002090,-96.418703,sklearn
2,0.001961,-96.418731,sklearn
3,0.002428,-96.418735,sklearn
0,0.059809,-99.119188,regain
1,0.052288,-96.704805,regain
2,0.059125,-96.447454,regain
3,0.077772,-96.421612,regain
0,0.014794,0.979997,admm
1,0.017387,0.998016,admm


In [46]:
#normalised accuracy [0,1]
norm_ac = np.array(df['accuracy'])
df['accuracy'] = (norm_ac - np.min(norm_ac))/np.ptp(norm_ac)
df

,time,accuracy,method
0,0.005095,0.026973,sklearn
1,0.002090,0.026973,sklearn
2,0.001961,0.026972,sklearn
3,0.002428,0.026972,sklearn
0,0.059809,0.000000,regain
1,0.052288,0.024115,regain
2,0.059125,0.026686,regain
3,0.077772,0.026944,regain
0,0.014794,0.999800,admm
1,0.017387,0.999980,admm


In [47]:
import plotly.express as px

fig = px.line(df, x="time", y="accuracy", color= "method", title='Perfomance comparison')
fig.show()

# Import LatentTimeGraphicalLasso

In [ ]:
from regain.covariance import LatentTimeGraphicalLasso

In [ ]:
from regain.covariance import latent_graphical_lasso_, latent_time_graphical_lasso_

In [ ]:
np.random.seed(243)

tau = 0.1 #mu1 in ADMM_SGL
alpha = 0.1 # lambda in ADMM_SGL

In [ ]:
sol = latent_graphical_lasso_.latent_graphical_lasso(
    true_cov, alpha=alpha, tau=tau, tol=1e-5, rtol=1e-5,
    rho=1. / true_cov.shape[0], verbose=0, max_iter=500)

In [ ]:
prec, cov, n_iter = sol[0], sol[1], sol[2]

In [ ]:
prec,

In [ ]:
cov